In [3]:
from keras.preprocessing.image import ImageDataGenerator
from os import listdir
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.constraints import maxnorm
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split


train_data_folder=['character_1_ka','character_2_kha','character_3_ga','character_4_gha','character_5_kna']
Label = ['Ka','Kha','Ga','Gha','Kna']
Labels = [0,1,2,3,4]


# In[19]:



    
def load_photos(train_data_folder,Desktop):
    i=0
    y=[]
    images=[]
    for directory in train_data_folder:
        y_val=Labels[i]
        
        for name in listdir(Desktop+'/'+ directory):
            # load an image from file
            filename = Desktop+'/'+directory + '/' + name
            image = load_img(filename) #target_size=(32, 32,1))
            # convert the image pixels to a numpy array
            image = img_to_array(image)
            # reshape data for the model
            image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
            # prepare the image for the VGG model
            image = preprocess_input(image)
            # get image id
            #image_id = name.split('.')[0]
            images.append(image)
            y.append(y_val)
        i=i+1;
    return images,y

# load images
Train_Desktop = 'Train_val'
images,y = load_photos(train_data_folder,Train_Desktop)
print('Loaded Images: %d' % len(images))





import numpy as np
x_train = np.asarray(images)
y_train = np.asarray(y)





datagen = ImageDataGenerator(
featurewise_center=True,
featurewise_std_normalization=True,
rotation_range=20,
width_shift_range=0.2,
height_shift_range=0.2,
horizontal_flip=True)

datagen.fit(x_train)
input_shape = (32, 32, 3)

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
# Creating a Sequential Model and adding the layers
model = Sequential()
model.add(Conv2D(128, kernel_size=(3,3), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(5,activation=tf.nn.softmax))


print(model.summary())


model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])



model.fit_generator(datagen.flow(x_train, y_train, batch_size=128),
                    steps_per_epoch=len(x_train) / 32, epochs=15,shuffle=True)



Desktop_test = 'Test'
images_test,y_test = load_photos(train_data_folder,Desktop_test)
print('Loaded Images Test: %d' % len(images_test))
import numpy as np
x_test = np.asarray(images_test)
y_test = np.asarray(y_test)
x_test = x_test / 255.0
datagen.fit(x_test)


# In[23]:


score_ADAM = model.evaluate_generator(datagen.flow(x_test, y_test, batch_size=32),steps=len(x_test)/32 )
print ("Accuracy with ADAM for Model 0 : " , score_ADAM[1])



Loaded Images: 8500
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3686528   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
Total params: 3,690,757
Trainable params: 3,690,757
Non-trainable params: 0
______________________________________________